In [6]:
import spacy, coreferee
from spacy.matcher import Matcher
import pandas as pd


In [7]:
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 3200000
nlp.add_pipe('coreferee')


In [24]:
import re
def import_file(file_name):
  text = ""


  with open(file_name, encoding="utf8") as f:
    try:
      text = f.readlines()
    except Exception as e:
      print(e)



  string_text = ""
  for line in text:
    string_text += line + "\n"
  string_text = re.sub('[\n]+', '\n', string_text)
  return string_text


In [1]:
filename = "LOTRC"

In [9]:
textString = import_file("./Datasets/"+filename+".txt")

In [10]:
textString[:1000]


'THE LORD\nOF THE RINGS\nChapter 1\nA LONG-EXPECTED PARTY\nWhen Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.\nBilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return. The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure. And if that was not enough for fame, there was also his prolonged vigour to marvel at. Time wore on, but it seemed to have little effect on Mr. Baggins. At ninety he was much the same as at fifty. At ninety-nine they began to call him well-preserved; but unchanged would have been nearer the mark. There were some that shook their heads and thought this was too much of a good t

In [11]:
doc = nlp(textString)


In [12]:
import pickle

pickle.dump(doc, open("./Dumps/LOTR/LOTRC_spacy.pkl", "wb"))

In [2]:
import pickle

In [3]:
coreference_resolved = ""
for document in doc:
  res = doc._.coref_chains.resolve(document)
  coreference_resolved += res[0].text +" " if res else document.text + " "


In [4]:
with open("./Datasets/"+filename+"_COREF.txt", "w", encoding="utf8") as f:
  f.write(coreference_resolved)

In [39]:
test = coreference_resolved[:500].split(".")

In [ ]:
test[:200]

In [22]:
textString = import_file("./Datasets/"+filename+"_COREF.txt")


In [24]:
textString[:100]

'\n Chapter 1 \n A LONG - EXPECTED PARTY \n When Mr. Bilbo Baggins of Bag End announced that Baggins wou'

In [15]:
doc = nlp(coreference_resolved)


In [1]:
import pickle

In [2]:
doc = pickle.load(open("./Dumps/LOTR/"+filename+"_COREF_spacy.pkl", "rb"))

In [5]:
len(doc)

677277

In [16]:
import pickle

pickle.dump(doc, open("./Dumps/LOTR/"+filename+"_COREF_spacy.pkl", "wb"))


In [3]:
newDoc = doc

In [4]:
pattern = [
    #First Pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"}, {"POS": {"IN": ["PROPN"]}, "OP":"+"}, 
    {"POS": "VERB"},     {"POS": "PART", "OP": "*"}, 
    {"POS": "ADV", "OP": "*"}, {"POS": {"IN": ["PROPN"]}, "OP": "+"},
    {"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"*"}],
    #Second pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"},
    {"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"},
     {"POS": {"IN": ["PART", "ADV", "CONJ"]}, "OP": "*"},  
     {"POS": {"IN": [ "ADV"]}, "OP": "+"},
     {"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"+"}],
    #Third pattern
    [{"POS": {"IN": ["PROPN"]}, "OP":"*"},
    {"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"},
     {"POS": {"IN": ["PART", "ADV", "CONJ"]}, "OP": "*"}, 
     {"POS": {"IN": ["ADV"]}, "OP": "+"}, 
     {"POS": {"IN": ["PROPN", ]}, "OP":"+"}, 
     {"POS": {"IN": ["PART", "ADV", "CONJ", "ADJ", "VERB"]}, "OP": "*"},  
     {"POS": {"IN": ["PROPN", "NOUN"]}, "OP":"+"}]
]


Try matching dependency tree

In [8]:
matcher = Matcher(nlp.vocab)
matcher.add("initial",pattern)


In [9]:
matches = matcher(newDoc)

In [12]:
def longest_string_array(array):
    longest_string = ""
    for string in array:
        if len(string) > len(longest_string):
            longest_string = string
    return longest_string

In [13]:
test_df = pd.DataFrame(columns=["Rel","End"])
filteredarray = []
otherArray = []
for match in matches:
  start = match[1]
  end = match[2]
  tempArray = []
  for new_match in matches:
    if new_match[1] == start or new_match[2] == end:
      tempArray.append(newDoc[new_match[1]:new_match[2]])
  if len(tempArray) > 1:
    sentence = longest_string_array(tempArray)

    if sentence.text != "" and sentence not in filteredarray:
      filteredarray.append(sentence)
  else:
    filteredarray.append(newDoc[start:end])
    test_df.loc[len(test_df)] = [newDoc[start:end], newDoc[end].i]


In [26]:
test_df = test_df.sort_values(by="End")

In [15]:
triples = pd.DataFrame(columns=["ENTITY1", "RELATION", "ENTITY2","ADDITIONAL"])
for relation in filteredarray:
  processed_relation = nlp(relation.text)
  i = 0
  entities = [False, False]
  relation = ""
  additional=""
  while i < len(processed_relation):
   
    prev_token = processed_relation[i-1] if i > 0 else None
    token = processed_relation[i]
    entity_name = ""
    if token.pos_ == "PROPN" or token.pos_ == "NOUN":
      
      entity_name += token.text
      j = i+1
      while j <len(processed_relation) and (processed_relation[j].pos_ == "PROPN" or processed_relation[j].pos_ == "NOUN"):
        entity_name += " " + processed_relation[j].text
        j += 1
      i = j-1
      if "subj" in processed_relation[i].dep_ and i < len(processed_relation):
        entities[0] = entity_name
      elif "obj" in processed_relation[i].dep_ and i < len(processed_relation):
        splitted = entity_name.split(" ")

        if len(splitted) > 1 and processed_relation[i].pos_ == "NOUN":
          entity_name = " ".join(splitted[:len(splitted)-1])
          additional = " ".join(splitted[len(splitted)-1:])
    
   
        
        entities[1] = entity_name
      
      
    else:
      relation +=" "+ token.text
    
    i+=1
  if len(entities) == 2 and entities[0] != False and entities[1] != False:
    splitted_second = entities[1].split(" ")
    if len(splitted_second) > 1 and entities[0] == splitted_second[0]:
      if entities[0] != entities[1]:
        triples.loc[len(triples)] = [entities[0], relation,
                                   entities[1],additional]
    else:
      if entities[0] != entities[1]:
        triples.loc[len(triples)] = [entities[0], relation, entities[1],additional]
  

Old --> compound --> PROPN
Odo --> compound --> PROPN
Proudfoot --> nsubj --> PROPN
removed --> ROOT --> VERB
Bilbo --> compound --> PROPN
feet --> dobj --> NOUN
Old --> compound --> PROPN
Odo --> compound --> PROPN
Proudfoot --> nsubj --> PROPN
removed --> ROOT --> VERB
Bilbo --> dobj --> PROPN


In [4]:
!py -3.9 -m pip3 install gensim

C:\Python39\python.exe: No module named pip3


In [30]:
from gensim.models import KeyedVectors
from collections.abc import Mapping
model = KeyedVectors.load('./word2vec-google-news-300.model')
model.vectors[1]


array([ 0.0703125 ,  0.08691406,  0.08789062,  0.0625    ,  0.06933594,
       -0.10888672, -0.08154297, -0.15429688,  0.02075195,  0.13183594,
       -0.11376953, -0.03735352,  0.06933594,  0.078125  , -0.10302734,
       -0.09765625,  0.04418945,  0.10253906, -0.06079102, -0.03613281,
       -0.04541016,  0.04736328, -0.12060547, -0.06396484,  0.0022583 ,
        0.03710938, -0.00291443,  0.11767578,  0.06176758,  0.06396484,
        0.08105469, -0.06884766, -0.0213623 ,  0.05517578, -0.08544922,
        0.06884766, -0.12792969, -0.03320312,  0.09863281,  0.17578125,
        0.11083984, -0.03466797, -0.04711914, -0.00848389,  0.03588867,
        0.10302734,  0.02697754, -0.02868652, -0.00512695,  0.10644531,
        0.05981445,  0.09423828,  0.03369141, -0.02709961, -0.09423828,
        0.00102997, -0.04833984,  0.03442383,  0.08105469, -0.11328125,
       -0.08886719,  0.03588867, -0.14550781, -0.24414062, -0.06152344,
        0.05297852,  0.05688477,  0.1796875 ,  0.06103516,  0.08

In [6]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec


In [1]:
import string


def simple_preproc(text):
  """
  see: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
  """
  return text.translate(str.maketrans('', '', string.punctuation))


In [7]:
with open("./Datasets/"+filename+'_COREF.txt', encoding="utf8") as read_file:
  sentences = [simple_preproc(k).lower().split()
               for k in read_file.readlines()]
model = Word2Vec(sentences=sentences, vector_size=100,
                 window=5, min_count=1, workers=4)


In [43]:
num_errors = 0
triples_vec = pd.DataFrame(columns=["ENTITY1", "RELATION", "ENTITY2","ADDITIONAL","VECTOR"])
for index, row in triples.iterrows():
  try:
    vectors=[]
    for word in row[1].split(" "):
      if word != '':
        vector = model.wv[word.strip()]
        vectors.append(vector)
    triples_vec.loc[len(triples_vec)] = [
        row[0], row[1], row[2], row[3], vectors]
  except Exception as e:
    num_errors +=1

"Key 'Merry' not present"
"Key 'Black' not present"
"Key 'Black' not present"
"Key 'Black' not present"
"Key 'Black' not present"
"Key 'Blessed' not present"
"Key 'High' not present"
"Key 'Last' not present"
"Key 'Dead' not present"
"Key 'Seven' not present"


In [40]:
triples_vec.head()

,ENTITY1,RELATION,ENTITY2,ADDITIONAL,VECTOR
0,Bilbo,adopted,Frodo,,-0.000392
1,Otho,disliked,Bilbo,,0.001169
2,Old Odo Proudfoot,removed,Bilbo,feet,0.005527
3,Old Odo Proudfoot,removed,Bilbo,,0.005527
4,Frodo,escorted,Lobelia,,0.000497


In [44]:
import numpy as np
temp_dataset = triples_vec
for index, row in triples_vec.iterrows():
  vectors = row[4]
  if len(vectors) > 1:
    average = np.mean(vectors, axis=0)
    temp_dataset.loc[index]["VECTOR"] = average
    
  else:
    temp_dataset.loc[index]["VECTOR"] = row[4][0]
temp_dataset.head(20)

,ENTITY1,RELATION,ENTITY2,ADDITIONAL,VECTOR
0,Bilbo,adopted,Frodo,,"[-0.04141598, 0.027508102, 0.03759231, -0.0180..."
1,Otho,disliked,Bilbo,,"[-0.016015012, 0.03036587, 0.015881192, -0.005..."
2,Old Odo Proudfoot,removed,Bilbo,feet,"[-0.16581842, 0.19962521, 0.2403444, 0.0179273..."
3,Old Odo Proudfoot,removed,Bilbo,,"[-0.16581842, 0.19962521, 0.2403444, 0.0179273..."
4,Frodo,escorted,Lobelia,,"[-0.002899614, 0.016226536, 0.0060042893, 0.01..."
5,Frodo,saw,Gandalf,,"[-0.810779, 0.08501016, 0.8626317, -0.16027759..."
6,Gandalf,paid,Frodo,,"[0.013581244, 0.23613575, -0.045862682, -0.111..."
7,Frodo,welcomed,Sam,,"[-0.06234623, 0.11143825, 0.048809897, -0.0334..."
8,Sam,pressed,Gandalf,,"[-0.07339541, 0.13966212, 0.11575856, 0.033556..."
9,Orcs,saw,Isildur,,"[-0.810779, 0.08501016, 0.8626317, -0.16027759..."


In [67]:
from sklearn.cluster import DBSCAN  # To instantiate and fit the model
from sklearn.metrics import pairwise_distances  # For Model evaluation
from sklearn.neighbors import NearestNeighbors  # For Hyperparameter Tuning
from sklearn.preprocessing import StandardScaler


In [107]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.05, min_samples=2).fit(temp_dataset["VECTOR"].values.tolist())  # fitting the model


In [113]:
unique = set(dbscan.labels_)

80

In [114]:
temp_dataset["LABEL"] = None

,ENTITY1,RELATION,ENTITY2,ADDITIONAL,VECTOR,LABEL
0,Bilbo,adopted,Frodo,,"[-0.04141598, 0.027508102, 0.03759231, -0.0180...",None
1,Otho,disliked,Bilbo,,"[-0.016015012, 0.03036587, 0.015881192, -0.005...",None
2,Old Odo Proudfoot,removed,Bilbo,feet,"[-0.16581842, 0.19962521, 0.2403444, 0.0179273...",None
3,Old Odo Proudfoot,removed,Bilbo,,"[-0.16581842, 0.19962521, 0.2403444, 0.0179273...",None
4,Frodo,escorted,Lobelia,,"[-0.002899614, 0.016226536, 0.0060042893, 0.01...",None


0

In [115]:
for index, row in temp_dataset.iterrows():
  temp_dataset.loc[index,"LABEL"] = dbscan.labels_[index]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [130]:
import random

for label in unique:
  current_label_items = temp_dataset.loc[temp_dataset["LABEL"] == label]
  relations = current_label_items["RELATION"].values
  average = np.mean(current_label_items["VECTOR"].values, axis=0)
  new_selected_relation = model.wv.most_similar(average)
  # print(model.wv.most_similar(average)[0][0])
  temp_dataset.loc[temp_dataset["LABEL"] == label,
                   "NEW_RELATION"] = new_selected_relation[0][0]
  


In [45]:
temp_dataset.to_csv("./Datasets/"+filename+"_TRIPLES.csv")
